In [ ]:
pip install -q datasets transformers

In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
dataset = load_dataset("AugustoReies/mascotas")

labels = dataset["test"].features["label"].names

In [ ]:
# Reemplaza 'nombre_de_la_columna_etiquetas' con el nombre real de la columna de etiquetas en tu conjunto de datos
etiquetas = dataset['test']

# Crea un arreglo para almacenar las etiquetas
y_true = []

#resultados
y_pred = []

# Almacena las etiquetas en el arreglo
for i in range(len(etiquetas)):
    label = etiquetas[i]['label']
    y_true.append(label)

In [ ]:
def evaluate_model(model_url):
  for i in range(len(etiquetas)):
      image = dataset["test"]["image"][i]

      image_processor = AutoImageProcessor.from_pretrained(model_url)
      model = ViTForImageClassification.from_pretrained(model_url)

      inputs = image_processor(image, return_tensors="pt")

      with torch.no_grad():
          logits = model(**inputs).logits

      predicted_label = logits.argmax(-1).item()
      y_pred.append(logits.argmax(-1)[0].item())

  # Calcular métricas
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred, average=None)
  recall = recall_score(y_true, y_pred, average=None)
  f1 = f1_score(y_true, y_pred, average=None)

  print("Accuracy:", accuracy)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1 Score:", f1)

In [ ]:
evaluate_model("AugustoReies/vit-base-patch16-224-mascotas")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(Y_1, Y_2, classes, figsize=(10, 10), text_size=12):
    # Creando la matriz de confusión
    cm = confusion_matrix(Y_1, Y_2)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]  # Normaliza la matriz de confusión
    n_classes = cm.shape[0]

    # Haciendo la matriz de confusión más visual
    fig, ax = plt.subplots(figsize=figsize)
    # Creando la gráfica de la matriz de confusión normalizada
    cax = ax.matshow(cm_norm, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    # Etiquetar los ejes
    ax.set(title="Mascotas Confusion Matrix",
           xlabel="Predicted label",
           ylabel="True label",
           xticks=np.arange(n_classes),
           yticks=np.arange(n_classes),
           xticklabels=classes,
           yticklabels=classes)

    # Poner los ejes X abajo
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    # Ajustar la fuente
    ax.yaxis.label.set_size(20)
    ax.xaxis.label.set_size(20)
    ax.title.set_size(20)

    # Ajustar el umbral para diferentes colores
    threshold = (cm.max() + cm.min()) / 2.

    # Graficar el texto en cada celda
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j] * 100:.1f}%)",
                 horizontalalignment="center",
                 color="white" if cm[i, j] > threshold else "black",
                 size=text_size)
    plt.show()

In [ ]:
plot_confusion_matrix(y_true, y_pred, labels)